In [2]:

import json
from pathlib import Path
from glob import glob
import os
from concurrent.futures import ProcessPoolExecutor
from itertools import chain
import nltk
import re
from tqdm import tqdm 

import text_utils as tu

## Download wikipedia data and extract it with wikiextractor


Download data from https://dumps.wikimedia.org/plwiki/20200301/

```
mkdir wiki_dump
cd wiki_dump
wget https://dumps.wikimedia.org/plwiki/20200301/plwiki-20200301-pages-articles-multistream1.xml-p1p169750.bz2
wget https://dumps.wikimedia.org/plwiki/20200301/plwiki-20200301-pages-articles-multistream2.xml-p169751p510662.bz2
wget https://dumps.wikimedia.org/plwiki/20200301/plwiki-20200301-pages-articles-multistream3.xml-p510663p1056310.bz2
wget https://dumps.wikimedia.org/plwiki/20200301/plwiki-20200301-pages-articles-multistream4.txt-p1056311p1831508.bz2
wget https://dumps.wikimedia.org/plwiki/20200301/plwiki-20200301-pages-articles-multistream5.xml-p1831509p3070393.bz2
wget https://dumps.wikimedia.org/plwiki/20200301/plwiki-20200301-pages-articles-multistream6.xml-p3070394p4570393.bz2
wget https://dumps.wikimedia.org/plwiki/20200301/plwiki-20200301-pages-articles-multistream6.xml-p4570394p4727706.bz2
```



```
#train.txt
plwiki-20200301-pages-articles-multistream1.xml-p1p169750
plwiki-20200301-pages-articles-multistream2.xml-p169751p510662
plwiki-20200301-pages-articles-multistream3.xml-p510663p1056310
plwiki-20200301-pages-articles-multistream4.xml-p1056311p1831508
plwiki-20200301-pages-articles-multistream5.xml-p1831509p3070393
plwiki-20200301-pages-articles-multistream6.xml-p3070394p4570393
```

```
#eval.txt
plwiki-20200301-pages-articles-multistream6.xml-p4570394p4727706
```

```
cd data/wiki_dump
cat train.txt | xargs -I@  python ../../libs/wikiextractor/WikiExtractor.py @ --bytes=100M --json --output="./train/@"
```

In [3]:

def process_line(line):
    try:
        doc = json.loads(line)
        txt = re.sub("\s+", " ", doc["text"])
        return txt
    except:
        # print(f"Could not parse line \n{line}\n")
        return ''

In [6]:

output_path='./data/corpus_raw/corpus_wikipedia_2020-03-01_train.txt'
wiki_json_files = [f for f in glob("data/wiki_dump/train/**", recursive=True) if os.path.isfile(f)]

with open(output_path, 'w+') as output_file:
    for json_line in tqdm(wiki_json_files):
        #print(f'process - {json_line}')

        with open(json_line) as f:
            text=''
            for line in tqdm(f,total=tu.get_num_lines(json_line)):
                text=process_line(line)

                output_file.write(text)
                # put new line of the end of the article
                output_file.write('\n\n')   


48/24248 [00:04<00:00, 6019.83it/s]
100%|██████████| 21/21 [03:09<00:00,  9.03s/it]
